<summary>Table of Contents</summary>

- [1. Standard Scaler Informer ](#1-standard-scaler-informer)
- [2. Standard Scaler PatchTST](#2-standard-scaler-patchtst)
- [3. MinMax Scaler Informer](#3-minmax-scaler-informer)
- [4. MinMax Scaler PatchTST](#4-minmax-scaler-patchtst)


Here we perform a check on **Germany** dataset to confirm choice of loss function and scaler for our data.

This script is to run the models. Final results are in the notebook "Comparison". 

Please note, the cell content is almost identical. However, when duplicating code and changing some arguments, it becomes easier to store and read results (especially if you want to experiment with 1 subpart) and split long running time into subprocesses. 

**For Standard Scaler and MinMax we tried learning rates: 0.0001, 0.00001, 0.000001.**

In [50]:
import os
import pandas as pd
import subprocess
import shutil
from utils.helper import extract_metrics_from_output, convert_results_into_df

# 1. Standard Scaler Informer

In [51]:
cuda_device = "1"

# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'DE_data.csv'
losses = ["MSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/standard"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [52]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
lr = "0.0001"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# Lists to store the results
informer_results_scaled, informer_results_unscaled = [], []

# Log file
with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to .ipynb output cell

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 48 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --dropout 0.1 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --overlapping_windows \
              --inverse \
              --loss_fnc "{loss}" \
              --scaler_type standard \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()
            
            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(informer_results_scaled, scaled_metrics), (informer_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_96_24_loss_choice_for_DE', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='standard', if_relu=False, channel_mixing=0, seq_len=96, label_len=48, pred_len=24, inverse=True, loss_fnc='MSE', fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.1, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience

In [ ]:
path_dir = './results/loss_fnc_choice'

if not os.path.exists(path_dir):
    os.makedirs(path_dir)

csv_name_scaled = 'informer_loss_functions_results_scaled_default.csv'
csv_name_unscaled = 'informer_loss_functions_results_unscaled_default.csv'

# Convert the results into a DataFrame and save as CSV
informer_df_scaled = convert_results_into_df(informer_results_scaled, path_dir, csv_name_scaled)
informer_df_unscaled = convert_results_into_df(informer_results_unscaled, path_dir, csv_name_unscaled)
informer_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.5247  0.7244  0.5114  0.5733
              2         24        0.5337  0.7306  0.4778  0.5782
              1         96        0.8678  0.9315  0.6915  0.7388
              2         96        0.9063  0.9520  0.6854  0.7550
              1         168       0.9473  0.9733  0.7125  0.7710
              2         168       1.0134  1.0067  0.7418  0.7975
MAE           1         24        0.5389  0.7341  0.4719  0.5810
              2         24        0.5458  0.7388  0.4761  0.5847
              1         96        0.9010  0.9492  0.6441  0.7529
              2         96        1.0010  1.0005  0.6780  0.7935
              1         168       0.9856  0.9928  0.7093  0.7864
              2         168       1.0407  1.0202  0.7039  0.8082

In [54]:
informer_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        21980930.0  4688.3823  3161.3560  0.2331
              2         24        21397382.0  4625.7305  2871.8848  0.2300
              1         96        38906908.0  6237.5400  4300.8379  0.3106
              2         96        40021996.0  6326.2939  4212.2036  0.3151
              1         168       42061940.0  6485.5176  4402.3633  0.3231
              2         168       45914860.0  6776.0503  4621.6470  0.3376
MAE           1         24        21871782.0  4676.7275  2841.9736  0.2325
              2         24        22062046.0  4697.0254  2876.5288  0.2335
              1         96        37380548.0  6113.9634  3884.0576  0.3045
              2         96        42881912.0  6548.4282  4106.7485  0.3261
              1         168       43904392.0  6626.0391  4388.9360  0.3301
              2         168       45537116.0  6748.1196  4292.6812  0.3362

In [ ]:
# Average the iterations
informer_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
informer_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

inf_res_scaled = informer_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
inf_res_unscaled = informer_unscaled.groupby(['Pred_len', 'Loss_function']).mean().sort_index().drop('Iteration', axis=1)
inf_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.5424  0.7365  0.4740  0.5829
         MSE            0.5292  0.7275  0.4946  0.5757
96       MAE            0.9510  0.9749  0.6610  0.7732
         MSE            0.8870  0.9418  0.6885  0.7469
168      MAE            1.0131  1.0065  0.7066  0.7973
         MSE            0.9804  0.9900  0.7271  0.7842

In [56]:
inf_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            21966914.0  4686.8765  2859.2512  0.2330
         MSE            21689156.0  4657.0564  3016.6204  0.2316
96       MAE            40131230.0  6331.1958  3995.4031  0.3153
         MSE            39464452.0  6281.9170  4256.5208  0.3128
168      MAE            44720754.0  6687.0793  4340.8086  0.3332
         MSE            43988400.0  6630.7839  4512.0051  0.3304

# 2. Standard Scaler PatchTST

In [57]:
# Dynamic + default variables
pred_lens = ["24", "96", "168"]
seq_len = "336"
lr = "0.0001"
model = "PatchTST"
itr = 2  
n_heads = "16"
d_model = "128"
d_ff = "256"
dropout = "0.2"
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"
                
            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers 3 \
              --factor 1 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --patch_len 32 \
              --stride 16 \
              --overlapping_windows \
              --inverse \
              --scaler_type standard \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_336_24_loss_choice_for_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='standard', if_relu=False, channel_mixing=0, seq_len=336, label_len=48, pred_len=24, inverse=True, loss_fnc='MSE', fc_dropout=0.2, head_dropout=0.0, patch_len=32, stride=16, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patien

In [ ]:
path_dir = './results/loss_fnc_choice'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_default.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_default.csv'
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.4391  0.6626  0.4261  0.5244
              2         24        0.4413  0.6643  0.4270  0.5258
              1         96        0.7834  0.8851  0.6169  0.7020
              2         96        0.7693  0.8771  0.6146  0.6956
              1         168       0.8303  0.9112  0.6474  0.7218
              2         168       0.8392  0.9161  0.6532  0.7257
MAE           1         24        0.4441  0.6664  0.4106  0.5274
              2         24        0.4470  0.6686  0.4129  0.5291
              1         96        0.7927  0.8903  0.5975  0.7061
              2         96        0.7960  0.8922  0.6009  0.7076
              1         168       0.8570  0.9258  0.6323  0.7334
              2         168       0.8853  0.9409  0.6346  0.7454

In [59]:
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        16458542.0  4056.9128  2476.0964  0.2017
              2         24        16442007.0  4054.8745  2471.8342  0.2016
              1         96        32124580.0  5667.8550  3648.3237  0.2823
              2         96        31628942.0  5623.9614  3646.1299  0.2801
              1         168       34333700.0  5859.4966  3841.3101  0.2919
              2         168       35108288.0  5925.2246  3900.4709  0.2952
MAE           1         24        16317905.0  4039.5427  2366.6226  0.2009
              2         24        16410974.0  4051.0461  2383.5229  0.2014
              1         96        32427548.0  5694.5190  3520.0559  0.2836
              2         96        32485964.0  5699.6460  3545.9673  0.2838
              1         168       35343512.0  5945.0410  3735.3179  0.2962
              2         168       36740496.0  6061.3940  3751.0430  0.3020

In [60]:
# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.4456  0.6675  0.4118  0.5283
         MSE            0.4402  0.6635  0.4266  0.5251
96       MAE            0.7943  0.8913  0.5992  0.7069
         MSE            0.7763  0.8811  0.6158  0.6988
168      MAE            0.8712  0.9333  0.6335  0.7394
         MSE            0.8348  0.9137  0.6503  0.7238

In [61]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            16364439.5  4045.2944  2375.0728  0.2011
         MSE            16450274.5  4055.8937  2473.9653  0.2017
96       MAE            32456756.0  5697.0825  3533.0116  0.2837
         MSE            31876761.0  5645.9082  3647.2268  0.2812
168      MAE            36042004.0  6003.2175  3743.1804  0.2991
         MSE            34720994.0  5892.3606  3870.8905  0.2936

In [62]:
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line

# Rename folder
os.rename("results_loss_unscaled", 'standard_unscaled')

# 3. MinMax Scaler Informer

We can use now "ReLU" activation function due to MinMax Scaler.

With BS 1036, ReLU - results are bad. (as twice as bad as with 32!)

In [63]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'DE_data.csv'
losses = ["MSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/min_max"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [64]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
lr = "0.0001"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# Lists to store the results
informer_results_scaled, informer_results_unscaled = [], []

# Log file
with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to .ipynb output cell

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Arguments for the command
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 48 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --dropout 0.1 \
              --overlapping_windows \
              --inverse \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()
            
            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(informer_results_scaled, scaled_metrics), (informer_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_96_24_loss_choice_for_DE', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=96, label_len=48, pred_len=24, inverse=True, loss_fnc='MSE', fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.1, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience=5,

In [ ]:
path_dir = './results/loss_fnc_choice'
csv_name_scaled = 'informer_loss_functions_results_scaled_minmax_default.csv'
csv_name_unscaled = 'informer_loss_functions_results_unscaled_minmax_default.csv'

# Convert the results into a DataFrame and save as CSV
informer_df_scaled = convert_results_into_df(informer_results_scaled, path_dir, csv_name_scaled)
informer_df_unscaled = convert_results_into_df(informer_results_unscaled, path_dir, csv_name_unscaled)
informer_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.0254  0.1594  0.1067  0.5630
              2         24        0.0244  0.1561  0.1057  0.5514
              1         96        0.0408  0.2021  0.1453  0.7157
              2         96        0.0411  0.2028  0.1442  0.7181
              1         168       0.0492  0.2219  0.1556  0.7862
              2         168       0.0482  0.2194  0.1538  0.7774
MAE           1         24        0.0250  0.1582  0.1025  0.5588
              2         24        0.0264  0.1624  0.1030  0.5734
              1         96        0.0412  0.2030  0.1423  0.7190
              2         96        0.0441  0.2101  0.1443  0.7440
              1         168       0.0530  0.2303  0.1560  0.8158
              2         168       0.0477  0.2183  0.1530  0.7735

In [66]:
informer_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        22144956.0  4705.8428  3012.8516  0.2340
              2         24        20262382.0  4501.3755  2949.7402  0.2238
              1         96        38047776.0  6168.2881  4162.5186  0.3072
              2         96        37857840.0  6152.8726  4103.2231  0.3064
              1         168       46881424.0  6847.0010  4476.5269  0.3412
              2         168       46423284.0  6813.4634  4453.2134  0.3395
MAE           1         24        21606588.0  4648.2886  2894.6770  0.2311
              2         24        22582330.0  4752.0869  2871.8965  0.2363
              1         96        38691036.0  6220.2119  4066.9163  0.3098
              2         96        41274512.0  6424.5244  4134.9688  0.3199
              1         168       50191892.0  7084.6235  4464.8423  0.3530
              2         168       45013488.0  6709.2090  4394.2524  0.3343

In [ ]:
# Average the iterations
informer_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
informer_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

inf_res_scaled = informer_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
inf_res_unscaled = informer_unscaled.groupby(['Pred_len', 'Loss_function']).mean().sort_index().drop('Iteration', axis=1)
inf_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.0257  0.1603  0.1028  0.5661
         MSE            0.0249  0.1578  0.1062  0.5572
96       MAE            0.0427  0.2066  0.1433  0.7315
         MSE            0.0410  0.2024  0.1447  0.7169
168      MAE            0.0503  0.2243  0.1545  0.7947
         MSE            0.0487  0.2207  0.1547  0.7818

In [68]:
# ALL 0.00001 - from 96 - BAD
# # 24 lr=0.000001; 96, 168 lr=0.00001 - BAD

inf_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            22094459.0  4700.1877  2883.2867  0.2337
         MSE            21203669.0  4603.6091  2981.2959  0.2289
96       MAE            39982774.0  6322.3682  4100.9425  0.3149
         MSE            37952808.0  6160.5803  4132.8708  0.3068
168      MAE            47602690.0  6896.9163  4429.5474  0.3436
         MSE            46652354.0  6830.2322  4464.8701  0.3403

# 4. MinMax Scaler PatchTST

In [69]:
log_dir = f"logs/loss_choice/min_max"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [70]:
# Dynamic variables
losses = ["MSE", "MAE"]
pred_lens = ["24", "96", "168"]
seq_len = "336"
lr = "0.0001"
model = "PatchTST"
itr = 2 
n_heads = "16"
d_model = "128"
d_ff = "256"
dropout = "0.2"
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers 3 \
              --factor 1 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --patch_len 32 \
              --stride 16 \
              --overlapping_windows \
              --inverse \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_336_24_loss_choice_for_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=336, label_len=48, pred_len=24, inverse=True, loss_fnc='MSE', fc_dropout=0.2, head_dropout=0.0, patch_len=32, stride=16, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience=

In [ ]:
path_dir = './results/loss_fnc_choice'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_default.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_default.csv'

# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.0209  0.1447  0.0911  0.5109
              2         24        0.0212  0.1456  0.0914  0.5141
              1         96        0.0362  0.1904  0.1299  0.6742
              2         96        0.0362  0.1902  0.1307  0.6734
              1         168       0.0392  0.1979  0.1380  0.7010
              2         168       0.0390  0.1976  0.1382  0.6999
MAE           1         24        0.0213  0.1458  0.0880  0.5148
              2         24        0.0214  0.1462  0.0884  0.5164
              1         96        0.0370  0.1924  0.1271  0.6812
              2         96        0.0372  0.1930  0.1280  0.6835
              1         168       0.0396  0.1990  0.1349  0.7051
              2         168       0.0405  0.2013  0.1346  0.7131

In [72]:
patchtst_results_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        16410582.0  4050.9976  2452.9761  0.2014
              2         24        16438184.0  4054.4031  2448.4897  0.2016
              1         96        31191604.0  5584.9443  3552.8628  0.2781
              2         96        31495776.0  5612.1099  3599.3784  0.2795
              1         168       34339608.0  5860.0005  3805.8254  0.2920
              2         168       34692824.0  5890.0615  3834.8921  0.2935
MAE           1         24        16238499.0  4029.7021  2347.3811  0.2004
              2         24        16319235.0  4039.7073  2364.4011  0.2009
              1         96        31829424.0  5641.7573  3474.2441  0.2810
              2         96        32093164.0  5665.0830  3500.1497  0.2821
              1         168       34746828.0  5894.6440  3705.5088  0.2937
              2         168       35872592.0  5989.3730  3699.5491  0.2984

In [ ]:
# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.0213  0.1460  0.0882  0.5156
         MSE            0.0211  0.1451  0.0912  0.5125
96       MAE            0.0371  0.1927  0.1275  0.6823
         MSE            0.0362  0.1903  0.1303  0.6738
168      MAE            0.0401  0.2002  0.1348  0.7091
         MSE            0.0391  0.1977  0.1381  0.7004

In [74]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            16278867.0  4034.7047  2355.8911  0.2006
         MSE            16424383.0  4052.7003  2450.7329  0.2015
96       MAE            31961294.0  5653.4202  3487.1969  0.2815
         MSE            31343690.0  5598.5271  3576.1206  0.2788
168      MAE            35309710.0  5942.0085  3702.5289  0.2961
         MSE            34516216.0  5875.0310  3820.3588  0.2927

In [75]:
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line

# Rename folder
os.rename("results_loss_unscaled", 'minmax')